In [1]:
import pandas as pd

# Load the dataset from a txt file
df = pd.read_csv('text03.txt', sep='\t')  

In [2]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Tokenize text and remove puncutation
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = word_tokenize(text.lower())
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

# Apply the cleaning function to the 'Text' column
df['Processed_Text'] = df['Text'].apply(clean_text)
print(df.head(5))



                                                Text  Label  \
0  Feeling unsupported by her team at work, she o...      0   
1  The joy of his son's return home after years a...      0   
2  Her close friend’s betrayal felt like a heavy ...      0   
3  Anticipation of the next day’s difficult conve...      0   
4  Financial disagreements with her spouse brough...      0   

                                      Processed_Text  
0  [feeling, unsupported, team, work, often, lay,...  
1  [joy, sons, return, home, years, abroad, disru...  
2  [close, friend, ’, betrayal, felt, like, heavy...  
3  [anticipation, next, day, ’, difficult, conver...  
4  [financial, disagreements, spouse, brought, la...  


In [3]:
from gensim.models import KeyedVectors

# Path to the GloVe file
glove_file = 'glove.6B.50d.txt'

# Load the GloVe model
def load_glove_model(glove_file):
    # Load the model as a Word2Vec format
    glove_model = KeyedVectors.load_word2vec_format(glove_file, binary=False, no_header=True)
    return glove_model

glove_vectors = load_glove_model(glove_file)


In [4]:
import numpy as np

def document_vector(doc):
    # Remove out-of-vocabulary words
    words = [word for word in doc if word in glove_vectors.key_to_index]
    if len(words) >= 1:
        return np.mean(glove_vectors[words], axis=0)
    else:
        return []

# Apply the function to each document
df['Text_Vector'] = df['Text'].apply(document_vector)


In [5]:
# Filter out empty entries
df = df[df['Text_Vector'].map(len) > 0]

# Extract features and labels
X = np.array(list(df['Text_Vector']))
y = np.array(df['Label'])

In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the SVM
svm_classifier = SVC(kernel='linear')  # You can experiment with different kernels
svm_classifier.fit(X_train, y_train)

# Evaluate the classifier
y_pred = svm_classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.00      0.00      0.00        16
           2       0.00      0.00      0.00         9
           3       0.00      0.00      0.00        21
           4       0.38      1.00      0.55        36

    accuracy                           0.38        95
   macro avg       0.08      0.20      0.11        95
weighted avg       0.14      0.38      0.21        95



C:\Users\chris\Documents\Python Projects\Sleep Classification\sleepCare\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\chris\Documents\Python Projects\Sleep Classification\sleepCare\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\chris\Documents\Python Projects\Sleep Classification\sleepCare\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

In [ ]:
print(X)